# データ取得(NVIDIA)

In [ ]:
%pip install torch tqdm pandas numpy yfinance

In [ ]:
import yfinance as yf
import pandas as pd

In [ ]:
#run if needed
# NVIDIAのティッカーシンボル（NVDA）
ticker = "NVDA"
# 20年分のデータを取得
data = yf.download(ticker, start="2004-01-01", end="2024-01-01")
# データを確認
print(data.head())
filename = "nvidia_20y_stock_data.csv"
data.to_csv(f"./RawData/{filename}")

# PreProcess

In [ ]:
#load csv file
FolderPath = "./RawData/nvidia_20y_stock_data.csv"
